##

## DB -> JSON 저장 - ver1

In [ ]:
import pymysql
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil import parser
import json
import os
from dotenv import load_dotenv

# 날짜 상수
START_DATE_STATIC = datetime(2000, 1, 1)
END_DATE_STATIC = datetime.today()

# .env 파일 로드
load_dotenv()

# DB 연결 함수
def get_connection():
    # .env 파일 필요
    host = os.getenv("DB_HOST")
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    db = os.getenv("DB_NAME")
    port = int(os.getenv("DB_PORT", "3306"))

    if not all([host, user, password, db]):
        raise ValueError("필수 DB 환경 변수가 .env에 누락되어 있습니다.")

    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        port=port,
        charset='utf8',
        connect_timeout=60,
        read_timeout=300,
        write_timeout=300
    )
    print(f"\nDB 연결 성공: {db}\n")
    return conn, db

# 테이블 목록 조회
def get_table_list(cursor, db_name):
    cursor.execute(f"SHOW TABLES FROM `{db_name}`")
    return [row[0] for row in cursor.fetchall()]

# datetime/timestamp/date 컬럼 조회
def get_datetime_column(cursor, db_name, table):
    cursor.execute("""
        SELECT COLUMN_NAME, DATA_TYPE
        FROM information_schema.columns
        WHERE table_schema = %s AND table_name = %s
    """, (db_name, table))
    datetime_cols = [row[0] for row in cursor.fetchall() if row[1].lower() in ("datetime", "timestamp", "date")]
    return datetime_cols[0] if datetime_cols else None

# 유효한 날짜 검사
def is_valid_datetime(value):
    return value not in ("0000-00-00", "0000-00-00 00:00:00", None, "")

# 날짜 범위 최소/최대 조회
def get_date_range(cursor, db_name, table, datetime_col):
    try:
        cursor.execute(f"""
            SELECT MIN(`{datetime_col}`), MAX(`{datetime_col}`)
            FROM `{db_name}`.`{table}`
            WHERE `{datetime_col}` IS NOT NULL
                AND `{datetime_col}` NOT IN ('0000-00-00', '0000-00-00 00:00:00')
                AND `{datetime_col}` >= '1900-01-01' AND `{datetime_col}` < '2100-01-01'
        """)
        result = cursor.fetchone()
        if not result or not is_valid_datetime(result[0]) or not is_valid_datetime(result[1]):
            return None, None
        return parser.parse(str(result[0])), parser.parse(str(result[1]))
    except:
        return None, None

# 평균 row 길이 조회
def get_avg_row_length(cursor, db_name, table):
    cursor.execute("""
        SELECT AVG_ROW_LENGTH
        FROM information_schema.tables
        WHERE table_schema = %s AND table_name = %s
    """, (db_name, table))
    res = cursor.fetchone()
    return res[0] if res and res[0] else 0

# 전체 row 수 조회
def get_row_count(cursor, db_name, table):
    cursor.execute(f"SELECT COUNT(*) FROM `{db_name}`.`{table}`")
    return cursor.fetchone()[0]

# 유효 날짜 기준 row 수 조회
def get_valid_row_count(cursor, db_name, table, datetime_col):
    cursor.execute(f"""
        SELECT COUNT(*)
        FROM `{db_name}`.`{table}`
        WHERE `{datetime_col}` IS NOT NULL
            AND `{datetime_col}` NOT IN ('0000-00-00', '0000-00-00 00:00:00')
            AND `{datetime_col}` >= '1900-01-01' AND `{datetime_col}` < '2100-01-01'
    """)
    return cursor.fetchone()[0]

# 단위별 날짜 범위 생성
def date_range_by_unit(start, end, unit):
    ranges = []
    current = start
    while current < end:
        if unit == 'week':
            next_point = current + timedelta(days=7)
            label = f"{current:%Y-W%U}"
        elif unit == 'month':
            next_point = current + relativedelta(months=1)
            label = f"{current:%Y-%m}"
        elif unit == 'year':
            next_point = current + relativedelta(years=1)
            label = f"{current:%Y}"
        else:
            raise ValueError("단위 오류")
        ranges.append((label, current, next_point))
        current = next_point
    return ranges

# 균등 분포 기반 용량 추정
def estimate_evenly_distributed_usage(row_count, avg_len, ranges):
    total_mb = row_count * avg_len / 1024 / 1024
    per_range_mb = total_mb / len(ranges) if ranges else 0
    return {label: f"{per_range_mb:,.3f}" for label, _, _ in ranges}, total_mb

# 날짜 컬럼 기준 용량 추정
def estimate_storage(cursor, db_name, table, datetime_col, start, end, avg_len):
    try:
        cursor.execute(f"""
            SELECT COUNT(*)
            FROM `{db_name}`.`{table}`
            WHERE `{datetime_col}` >= %s AND `{datetime_col}` < %s
                AND `{datetime_col}` NOT IN ('0000-00-00', '0000-00-00 00:00:00')
        """, (start, end))
        count = cursor.fetchone()[0]
        return round(count * avg_len / 1024 / 1024, 3), count
    except:
        return 0, 0

# 날짜 형식 안전 포맷
def safe_date_format(date_obj):
    return date_obj.strftime('%Y-%m-%d') if hasattr(date_obj, 'strftime') else str(date_obj)

# 메인 처리 함수
def main():
    conn, db_name = get_connection()
    cursor = conn.cursor()
    tables = get_table_list(cursor, db_name)

    result = {
        "총 테이블 수": len(tables),
        "총 테이블 명 목록": tables,
        "날짜 컬럼이 있는 테이블 수": 0,
        "날짜 컬럼이 없는 테이블 수": 0,
        "데이터가 없는 테이블 수": 0,
        "평균 row length가 0인 테이블 수": 0,
        "건너뛴 테이블 수": 0,
        "건너뛴 테이블 목록": [],
        "평균 row length가 0인 테이블 목록": [],
        "데이터가 없는 테이블 목록": [],
        "날짜 컬럼이 있는 테이블 명 목록": [],
        "날짜 컬럼이 없는 테이블 명 목록": [],
        "날짜 컬럼이 있는 테이블": {},
        "날짜 컬럼이 없는 테이블": {},
        "데이터가 없는 테이블": {}
    }

    exclude_tables = []
    skipped_tables = []

    for table in tables:
        if table in exclude_tables:
            print(f"건너뜀: {table}")
            skipped_tables.append(table)
            result["건너뛴 테이블 수"] += 1
            continue

        try:
            print(f"분석 중: {table}")
            row_count = get_row_count(cursor, db_name, table)
            avg_len = get_avg_row_length(cursor, db_name, table)

            # 데이터 없음
            if row_count == 0:
                result["데이터가 없는 테이블 수"] += 1
                result["데이터가 없는 테이블 목록"].append(table)
                result["데이터가 없는 테이블"][table] = {
                    "startDate": None,
                    "endDate": None,
                    "week": {},
                    "month": {},
                    "year": {}
                }
                continue

            # 평균 길이 0
            if avg_len == 0:
                result["평균 row length가 0인 테이블 수"] += 1
                result["평균 row length가 0인 테이블 목록"].append(table)
                continue

            datetime_col = get_datetime_column(cursor, db_name, table)

            if datetime_col:
                # --- datetime 컬럼이 있을 때 처리 ---
                start_date, end_date = get_date_range(cursor, db_name, table, datetime_col)
                valid_count = get_valid_row_count(cursor, db_name, table, datetime_col)
                invalid_count = row_count - valid_count

                if not start_date or not end_date:
                    valid_count = 0
                    invalid_count = row_count

                result["날짜 컬럼이 있는 테이블 수"] += 1
                result["날짜 컬럼이 있는 테이블 명 목록"].append(table)
                table_result = {
                    "startDate": safe_date_format(start_date or START_DATE_STATIC),
                    "endDate": safe_date_format(end_date or END_DATE_STATIC),
                    "week": {},
                    "month": {},
                    "year": {}
                }

                for unit in ['week', 'month', 'year']:
                    total_unit_mb = 0
                    unit_result = {}
                    for label, s, e in date_range_by_unit(start_date or START_DATE_STATIC, end_date or END_DATE_STATIC, unit):
                        mb, count = estimate_storage(cursor, db_name, table, datetime_col, s, e, avg_len)
                        if count > 0:
                            unit_result[label] = f"{mb:,.3f}"
                            total_unit_mb += mb

                    # 비정상 row 균등 처리
                    if invalid_count > 0:
                        invalid_mb = invalid_count * avg_len / 1024 / 1024
                        unit_result["기타"] = f"{invalid_mb:,.3f}"

                    table_result[unit] = unit_result

                result["날짜 컬럼이 있는 테이블"][table] = table_result

            else:
                # --- datetime 컬럼이 없을 때 균등 분포 처리 ---
                result["날짜 컬럼이 없는 테이블 수"] += 1
                result["날짜 컬럼이 없는 테이블 명 목록"].append(table)

                # 주·월·년 범위 생성
                week_ranges  = date_range_by_unit(START_DATE_STATIC, END_DATE_STATIC, 'week')
                month_ranges = date_range_by_unit(START_DATE_STATIC, END_DATE_STATIC, 'month')
                year_ranges  = date_range_by_unit(START_DATE_STATIC, END_DATE_STATIC, 'year')

                # 균등 분포로 사용량 추정
                week_usage,  _ = estimate_evenly_distributed_usage(row_count, avg_len, week_ranges)
                month_usage, _ = estimate_evenly_distributed_usage(row_count, avg_len, month_ranges)
                year_usage,  _ = estimate_evenly_distributed_usage(row_count, avg_len, year_ranges)

                result["날짜 컬럼이 없는 테이블"][table] = {
                    "startDate": safe_date_format(START_DATE_STATIC),
                    "endDate":   safe_date_format(END_DATE_STATIC),
                    "week":  week_usage,
                    "month": month_usage,
                    "year":  year_usage
                }
                continue

        except Exception as e:
            print(f"오류 발생: {table} → {e}")
            continue

    # 최종 결과 설정
    result["건너뛴 테이블 목록"] = skipped_tables

    # JSON 저장
    now_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    json_file = f"db_storage_summary_{now_str}.json"
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=2, ensure_ascii=False)

    print("\n결과 요약:")
    print(json.dumps(result, indent=2, ensure_ascii=False))
    print(f"\nJSON 저장 위치: {os.path.abspath(json_file)}")

    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()

## DB -> JSON 저장 - ver2

In [ ]:
import pymysql
from datetime import datetime
from dateutil import parser
import json
import os
from dotenv import load_dotenv

# 날짜 상수
START_DATE_STATIC = datetime(2000, 1, 1)
END_DATE_STATIC = datetime.today()

# .env 파일 로드
load_dotenv()

# DB 연결
def get_connection():
    host = os.getenv("DB_HOST")
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    db = os.getenv("DB_NAME")
    port = int(os.getenv("DB_PORT", "3306"))

    if not all([host, user, password, db]):
        raise ValueError("필수 DB 환경 변수가 .env에 누락되어 있습니다.")

    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        port=port,
        charset='utf8',
        connect_timeout=60,
        read_timeout=300,
        write_timeout=300
    )
    print(f"\nDB 연결 성공: {db}\n")
    return conn, db

# 테이블 목록 조회 (뷰 제외)
def get_table_list(cursor, db_name):
    cursor.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = %s
            AND table_type = 'BASE TABLE'
    """, (db_name,))
    return [row[0] for row in cursor.fetchall()]

# datetime 컬럼 조회
def get_datetime_column(cursor, db_name, table):
    cursor.execute("""
        SELECT COLUMN_NAME, DATA_TYPE
        FROM information_schema.columns
        WHERE table_schema=%s AND table_name=%s
    """, (db_name, table))
    for col, dtype in cursor.fetchall():
        if dtype.lower() in ('datetime','timestamp','date'):
            return col
    return None

# 평균 row 길이 조회
def get_avg_row_length(cursor, db_name, table):
    cursor.execute("""
        SELECT AVG_ROW_LENGTH
        FROM information_schema.tables
        WHERE table_schema=%s AND table_name=%s
    """, (db_name, table))
    result = cursor.fetchone()
    return result[0] if result and result[0] else 0

# 전체 row 수 조회
def get_row_count(cursor, db_name, table):
    cursor.execute(f"SELECT COUNT(*) FROM `{db_name}`.`{table}`")
    return cursor.fetchone()[0]

# 주/월/연도별 분포 조회 (1회 스캔)
def get_distribution(cursor, db_name, table, datetime_col, unit):
    if unit=='year':
        period_sql = "YEAR(`{col}`)"
        label_fmt = lambda x: str(x)
    elif unit=='month':
        period_sql = "DATE_FORMAT(`{col}`, '%Y-%m')"
        label_fmt = lambda x: x
    elif unit=='week':
        period_sql = "YEARWEEK(`{col}`,3)"
        label_fmt = lambda x: str(x)
    else:
        raise ValueError("단위 오류")
    col = datetime_col
    sql = f"""
        SELECT {period_sql.replace('{col}',col)} AS period, COUNT(*) AS cnt
        FROM `{db_name}`.`{table}`
        WHERE `{col}` NOT IN ('0000-00-00','0000-00-00 00:00:00')
            AND `{col}` BETWEEN '1900-01-01' AND '2100-01-01'
        GROUP BY period
    """
    cursor.execute(sql)
    dist = {}
    for period, cnt in cursor.fetchall():
        label = label_fmt(period)
        dist[label] = cnt
    return dist

# 안전한 날짜 포맷
def safe_date_format(dt):
    return dt.strftime('%Y-%m-%d') if hasattr(dt,'strftime') else str(dt)

# 메인 함수
def main():
    conn, db_name = get_connection()
    cursor = conn.cursor()
    tables = get_table_list(cursor, db_name)

    result = {
        "총 테이블 수": len(tables),
        "총 테이블 명 목록": tables,
        "날짜 컬럼이 있는 테이블 수": 0,
        "날짜 컬럼이 없는 테이블 수": 0,
        "데이터가 없는 테이블 수": 0,
        "평균 row length가 0인 테이블 수": 0,
        "건너뛴 테이블 수": 0,
        "건너뛴 테이블 목록": [],
        "평균 row length가 0인 테이블 목록": [],
        "날짜 컬럼이 있는 테이블 명 목록": [],
        "날짜 컬럼이 없는 테이블 명 목록": [],
        "데이터가 없는 테이블 목록": [],
        "날짜 컬럼이 있는 테이블": {},
        "날짜 컬럼이 없는 테이블": {},
        "데이터가 없는 테이블": {}
    }

    for table in tables:
        # 현재 분석 중인 테이블 출력
        print(f"분석 중 테이블: {table}")
        row_count = get_row_count(cursor, db_name, table)
        avg_len = get_avg_row_length(cursor, db_name, table)

        if row_count == 0:
            result["데이터가 없는 테이블 수"] += 1
            result["데이터가 없는 테이블 목록"].append(table)
            result["데이터가 없는 테이블"][table] = {"startDate": None, "endDate": None, "week": {}, "month": {}, "year": {}}
            continue

        if avg_len == 0:
            result["평균 row length가 0인 테이블 수"] += 1
            result["평균 row length가 0인 테이블 목록"].append(table)
            continue

        datetime_col = get_datetime_column(cursor, db_name, table)
        if not datetime_col:
            # 날짜 컬럼 없음 → 균등 분포 처리
            result["날짜 컬럼이 없는 테이블 수"] += 1
            result["날짜 컬럼이 없는 테이블 명 목록"].append(table)
            from dateutil.relativedelta import relativedelta
            from datetime import timedelta
            def ranges(start, end, unit):
                lst = []
                cur = start
                while cur < end:
                    if unit == 'week':
                        nxt = cur + timedelta(days=7)
                        lbl = f"{cur:%Y-W%U}"
                    elif unit == 'month':
                        nxt = cur + relativedelta(months=1)
                        lbl = f"{cur:%Y-%m}"
                    else:
                        nxt = cur + relativedelta(years=1)
                        lbl = f"{cur:%Y}"
                    lst.append((lbl, cur, nxt))
                    cur = nxt
                return lst
            table_res = {"startDate": safe_date_format(START_DATE_STATIC), "endDate": safe_date_format(END_DATE_STATIC)}
            for unit in ['week', 'month', 'year']:
                rngs = ranges(START_DATE_STATIC, END_DATE_STATIC, unit)
                total_mb = row_count * avg_len / 1024 / 1024
                per_mb = total_mb / len(rngs) if rngs else 0
                table_res[unit] = {lbl: f"{per_mb:,.3f}" for lbl, _, _ in rngs}
            result["날짜 컬럼이 없는 테이블"][table] = table_res
            continue

        # datetime 컬럼 존재
        result["날짜 컬럼이 있는 테이블 수"] += 1
        result["날짜 컬럼이 있는 테이블 명 목록"].append(table)

        # 전체 valid 분포 조회
        dist_year  = get_distribution(cursor, db_name, table, datetime_col, 'year')
        dist_month = get_distribution(cursor, db_name, table, datetime_col, 'month')
        dist_week  = get_distribution(cursor, db_name, table, datetime_col, 'week')
        valid_count   = sum(dist_year.values())
        invalid_count = row_count - valid_count

        # 기간별 용량 계산
        table_res = {'startDate': None, 'endDate': None, 'week': {}, 'month': {}, 'year': {}}
        cursor.execute(f"SELECT MIN(`{datetime_col}`), MAX(`{datetime_col}`) "
                        f"FROM `{db_name}`.`{table}` "
                        f"WHERE `{datetime_col}` NOT IN ('0000-00-00','0000-00-00 00:00:00')")
        sd, ed = cursor.fetchone()
        table_res['startDate'] = safe_date_format(parser.parse(str(sd))) if sd else safe_date_format(START_DATE_STATIC)
        table_res['endDate']   = safe_date_format(parser.parse(str(ed))) if ed else safe_date_format(END_DATE_STATIC)
        for unit, dist in (('year', dist_year), ('month', dist_month), ('week', dist_week)):
            ures = {lbl: f"{cnt * avg_len / 1024 / 1024:,.3f}" for lbl, cnt in dist.items()}
            if invalid_count > 0:
                ures['기타'] = f"{invalid_count * avg_len / 1024 / 1024:,.3f}"
            table_res[unit] = ures
        result["날짜 컬럼이 있는 테이블"][table] = table_res

    # 결과 JSON 저장
    jsf = f"db_storage_summary_{datetime.now():%Y%m%d_%H%M%S}.json"
    with open(jsf, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    print(f"완료: {jsf}")

    cursor.close()
    conn.close()

if __name__ == '__main__':
    main()

## JSON -> Jupyter Notebook 활용(Export to HTML) - ver1

In [ ]:
import glob
import json
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime

pio.renderers.default = "notebook_connected"  # 또는 "iframe_connected"

# ───────────────────────────────────────────────────────────────
# 1. JSON 파일 찾기
json_files = sorted(glob.glob("*.json"), reverse=True)
if not json_files:
    raise FileNotFoundError("*.json 파일을 찾을 수 없습니다.")
json_path = json_files[0]
print(f"▶ 로드할 JSON: {json_path}")

# 2. JSON 불러오기
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ───────────────────────────────────────────────────────────────
# 3. 전체 요약 출력 (표로 표현) 및 용량 총합 계산
summary_keys = [
    "총 테이블 수",
    "날짜 컬럼이 있는 테이블 수",
    "날짜 컬럼이 없는 테이블 수",
    "데이터가 없는 테이블 수",
    "건너뛴 테이블 수",
    "평균 row length가 0인 테이블 수"
]

tables_with_date = data.get("날짜 컬럼이 있는 테이블", {})
sum_with_date_mb = sum(
    sum(float(v.replace(",", "")) for v in content.get("year", {}).values())
    for content in tables_with_date.values()
)

tables_without_date = data.get("날짜 컬럼이 없는 테이블", {})
sum_without_date_mb = sum(
    sum(float(v.replace(",", "")) for v in content.get("year", {}).values())
    for content in tables_without_date.values()
)

summary_keys += [
    "날짜 컬럼이 있는 테이블 용량 총합(MB)",
    "날짜 컬럼이 없는 테이블 용량 총합(MB)",
    "총 합(MB)"
]
summary_values = [data.get(key, 0) for key in summary_keys[:-3]] + [
    f"{sum_with_date_mb:,.2f}",
    f"{sum_without_date_mb:,.2f}",
    f"{(sum_with_date_mb + sum_without_date_mb):,.2f}"
]

fig_summary = go.Figure(data=[go.Table(
    header=dict(values=summary_keys, align="center", fill_color="lightgray"),
    cells=dict(values=[[v] for v in summary_values], align="center")
)])
fig_summary.update_layout(
    title=dict(text="전체 요약", x=0, xanchor='left'),
    width=len(summary_keys) * 200
)
display(HTML(pio.to_html(fig_summary, full_html=False, include_plotlyjs='cdn')))

# ───────────────────────────────────────────────────────────────
# 4. 날짜 컬럼이 있는 테이블 – 연도별 라인차트
fig1 = go.Figure()
table_names_1, totals_1 = [], []

all_year_labels = set()
for content in tables_with_date.values():
    all_year_labels.update(content.get("year", {}).keys())
others = sorted([y for y in all_year_labels if y != "기타"])
full_order = others + (["기타"] if "기타" in all_year_labels else [])

for tbl, content in tables_with_date.items():
    year_data = content.get("year", {})
    if not year_data:
        continue
    x = [y for y in full_order if y in year_data]
    y = [float(year_data[y].replace(",", "")) for y in x]

    fig1.add_trace(go.Scatter(x=x, y=y, mode="lines+markers", name=tbl))
    table_names_1.append(tbl)
    totals_1.append(f"{sum(y):,.2f}")

fig1.update_layout(
    title=dict(text="날짜 컬럼이 있는 테이블 – 연도별 용량 추이", x=0, xanchor='left'),
    xaxis_title="연도",
    yaxis_title="용량 (MB)",
    template="plotly_white",
    xaxis=dict(categoryorder="array", categoryarray=full_order, tickangle=0)
)
display(HTML(pio.to_html(fig1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 5. 날짜 컬럼이 있는 테이블 – 총합 용량 요약표 (MB)
if table_names_1:
    fig_table1 = go.Figure(data=[go.Table(
        header=dict(values=table_names_1, fill_color="lightblue", align="center"),
        cells=dict(values=[[v] for v in totals_1], align="center")
    )])
    fig_table1.update_layout(
        title=dict(text="날짜 컬럼이 있는 테이블 – 총합 용량 (MB)", x=0, xanchor='left'),
        width=len(table_names_1) * 200
    )
    display(HTML(pio.to_html(fig_table1, full_html=False, include_plotlyjs=False)))

    # 5a. 날짜 컬럼이 있는 테이블 – 용량 순 (MB) 정렬표
    pairs1 = sorted(zip(table_names_1, totals_1),
                    key=lambda x: float(x[1].replace(",", "")),
                    reverse=True)
    names_rank1, vals_rank1 = zip(*pairs1)
    fig_rank1 = go.Figure(data=[go.Table(
        header=dict(values=list(names_rank1), fill_color="lavender", align="center"),
        cells=dict(values=[[v] for v in vals_rank1], align="center")
    )])
    fig_rank1.update_layout(
        title=dict(text="날짜 컬럼이 있는 테이블 – 용량 순 (MB)", x=0, xanchor='left'),
        width=len(names_rank1) * 200
    )
    display(HTML(pio.to_html(fig_rank1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 6. 날짜 컬럼이 있는 테이블 – 월 전체 데이터 표 (MB)
if table_names_1:
    months = sorted({m for content in tables_with_date.values() for m in content.get("month", {})})
    header_month1 = ["월"] + table_names_1
    cells_month1 = [months] + [
        [tables_with_date[tbl].get("month", {}).get(m, "0.000") for m in months]
        for tbl in table_names_1
    ]
    fig_month1 = go.Figure(data=[go.Table(
        header=dict(values=header_month1, align="center"),
        cells=dict(values=cells_month1, align="center")
    )])
    fig_month1.update_layout(
        title=dict(text="날짜 컬럼이 있는 테이블 – 월 전체 데이터 (MB)", x=0, xanchor='left'),
        width=len(header_month1) * 200
    )
    display(HTML(pio.to_html(fig_month1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 7. 날짜 컬럼이 있는 테이블 – 연도 전체 데이터 표 (MB)
if table_names_1:
    years_all = sorted({y for content in tables_with_date.values() for y in content.get("year", {})})
    header_year1 = ["연도"] + table_names_1
    cells_year1 = [years_all] + [
        [tables_with_date[tbl].get("year", {}).get(y, "0.000") for y in years_all]
        for tbl in table_names_1
    ]
    fig_year1 = go.Figure(data=[go.Table(
        header=dict(values=header_year1, align="center"),
        cells=dict(values=cells_year1, align="center")
    )])
    fig_year1.update_layout(
        title=dict(text="날짜 컬럼이 있는 테이블 – 연도 전체 데이터 (MB)", x=0, xanchor='left'),
        width=len(header_year1) * 200
    )
    display(HTML(pio.to_html(fig_year1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 8. 날짜 컬럼이 없는 테이블 – 연도별 라인차트
fig2 = go.Figure()
table_names_2, totals_2 = [], []

for tbl, content in tables_without_date.items():
    year_data = content.get("year", {})
    if not year_data:
        continue
    years2 = sorted([k for k in year_data.keys() if k != "기타"])
    if "기타" in year_data:
        years2.append("기타")
    x2 = years2
    y2 = [float(year_data[k].replace(",", "")) for k in x2]

    fig2.add_trace(go.Scatter(x=x2, y=y2, mode="lines+markers", name=tbl))
    table_names_2.append(tbl)
    totals_2.append(f"{sum(y2):,.2f}")

fig2.update_layout(
    title=dict(text="날짜 컬럼이 없는 테이블 – 연도별 용량 추이", x=0, xanchor='left'),
    xaxis_title="연도",
    yaxis_title="용량 (MB)",
    template="plotly_white",
    xaxis=dict(categoryorder="array", categoryarray=years2, tickangle=0)
)
display(HTML(pio.to_html(fig2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 9. 날짜 컬럼이 없는 테이블 – 총합 용량 요약표 (MB)
if table_names_2:
    fig_table2 = go.Figure(data=[go.Table(
        header=dict(values=table_names_2, fill_color="lightgray", align="center"),
        cells=dict(values=[[v] for v in totals_2], align="center")
    )])
    fig_table2.update_layout(
        title=dict(text="날짜 컬럼이 없는 테이블 – 총합 용량 (MB)", x=0, xanchor='left'),
        width=len(table_names_2) * 200
    )
    display(HTML(pio.to_html(fig_table2, full_html=False, include_plotlyjs=False)))

    # 9a. 날짜 컬럼이 없는 테이블 – 용량 순 (MB) 정렬표
    pairs2 = sorted(zip(table_names_2, totals_2),
                    key=lambda x: float(x[1].replace(",", "")),
                    reverse=True)
    names_rank2, vals_rank2 = zip(*pairs2)
    fig_rank2 = go.Figure(data=[go.Table(
        header=dict(values=list(names_rank2), fill_color="lavender", align="center"),
        cells=dict(values=[[v] for v in vals_rank2], align="center")
    )])
    fig_rank2.update_layout(
        title=dict(text="날짜 컬럼이 없는 테이블 – 용량 순 (MB)", x=0, xanchor='left'),
        width=len(names_rank2) * 200
    )
    display(HTML(pio.to_html(fig_rank2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 10. 날짜 컬럼이 없는 테이블 – 월 전체 데이터 표 (MB)
if table_names_2:
    months2 = sorted({m for content in tables_without_date.values() for m in content.get("month", {})})
    header_month2 = ["월"] + table_names_2
    cells_month2 = [months2] + [
        [tables_without_date[tbl].get("month", {}).get(m, "0.000") for m in months2]
        for tbl in table_names_2
    ]
    fig_month2 = go.Figure(data=[go.Table(
        header=dict(values=header_month2, align="center"),
        cells=dict(values=cells_month2, align="center")
    )])
    fig_month2.update_layout(
        title=dict(text="날짜 컬럼이 없는 테이블 – 월 전체 데이터 (MB)", x=0, xanchor='left'),
        width=len(header_month2) * 200
    )
    display(HTML(pio.to_html(fig_month2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 11. 날짜 컬럼이 없는 테이블 – 연도 전체 데이터 표 (MB)
if table_names_2:
    years2_all = sorted({y for content in tables_without_date.values() for y in content.get("year", {})})
    header_year2 = ["연도"] + table_names_2
    cells_year2 = [years2_all] + [
        [tables_without_date[tbl].get("year", {}).get(y, "0.000") for y in years2_all]
        for tbl in table_names_2
    ]
    fig_year2 = go.Figure(data=[go.Table(
        header=dict(values=header_year2, align="center"),
        cells=dict(values=cells_year2, align="center")
    )])
    fig_year2.update_layout(
        title=dict(text="날짜 컬럼이 없는 테이블 – 연도 전체 데이터 (MB)", x=0, xanchor='left'),
        width=len(header_year2) * 200
    )
    display(HTML(pio.to_html(fig_year2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 12. 데이터가 없는 테이블 – 표로 표현
empty_tables = data.get("데이터가 없는 테이블 목록", [])
if empty_tables:
    fig_empty = go.Figure(data=[go.Table(
        cells=dict(values=empty_tables, align="center")
    )])
    fig_empty.update_layout(
        title=dict(text="데이터가 없는 테이블 목록", x=0, xanchor='left'),
        width=len(empty_tables) * 200
    )
    display(HTML(pio.to_html(fig_empty, full_html=False, include_plotlyjs=False)))
else:
    print("\n=== 데이터가 없는 테이블 목록 ===\n없음")

## JSON -> Jupyter Notebook 활용(Export to HTML) - ver2

In [ ]:
import glob
import json
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime

pio.renderers.default = "notebook_connected"  # 또는 "iframe_connected"

# ───────────────────────────────────────────────────────────────
# 1. JSON 파일 찾기
json_files = sorted(glob.glob("*.json"), reverse=True)
if not json_files:
    raise FileNotFoundError("*.json 파일을 찾을 수 없습니다.")
json_path = json_files[0]
print(f"▶ 로드할 JSON: {json_path}")

# 2. JSON 불러오기
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ───────────────────────────────────────────────────────────────
# 3. 전체 요약 출력 (표 세로 배치) 및 용량 총합 계산
summary_keys = [
    "총 테이블 수",
    "날짜 컬럼이 있는 테이블 수",
    "날짜 컬럼이 없는 테이블 수",
    "데이터가 없는 테이블 수"
]

tables_with_date = data.get("날짜 컬럼이 있는 테이블", {})
sum_with_date_mb = sum(
    sum(float(v.replace(",", "")) for v in content.get("year", {}).values())
    for content in tables_with_date.values()
)

tables_without_date = data.get("날짜 컬럼이 없는 테이블", {})
sum_without_date_mb = sum(
    sum(float(v.replace(",", "")) for v in content.get("year", {}).values())
    for content in tables_without_date.values()
)

summary_keys += [
    "날짜 컬럼이 있는 테이블 용량 총합(MB)",
    "날짜 컬럼이 없는 테이블 용량 총합(MB)",
    "총 합(MB)"
]
summary_values = [data.get(key, 0) for key in summary_keys[:-3]] + [
    f"{sum_with_date_mb:,.2f}",
    f"{sum_without_date_mb:,.2f}",
    f"{(sum_with_date_mb + sum_without_date_mb):,.2f}"
]

fig_summary = go.Figure(data=[go.Table(
    header=dict(
        values=["항목", "값"],
        align="center",
        fill_color="lightgray",
        line_color="black",
        height=40
    ),
    cells=dict(
        values=[summary_keys, summary_values],
        align="center",
        line_color="black",
        height=30
    )
)])
fig_summary.update_layout(
    title=dict(text="전체 요약", x=0, xanchor='left'),
    width=600,
    height=450
)
display(HTML(pio.to_html(fig_summary, full_html=False, include_plotlyjs='cdn')))

# ───────────────────────────────────────────────────────────────
# 4. 날짜 컬럼이 있는 테이블 – 연도별 라인차트
fig1 = go.Figure()
table_names_1, totals_1 = [], []

all_year_labels = set()
for content in tables_with_date.values():
    all_year_labels.update(content.get("year", {}).keys())
others = sorted([y for y in all_year_labels if y != "기타"])
full_order = others + (["기타"] if "기타" in all_year_labels else [])

for tbl, content in tables_with_date.items():
    year_data = content.get("year", {})
    if not year_data:
        continue
    x = [y for y in full_order if y in year_data]
    y = [float(year_data[y].replace(",", "")) for y in x]

    fig1.add_trace(go.Scatter(x=x, y=y, mode="lines+markers", name=tbl))
    table_names_1.append(tbl)
    totals_1.append(f"{sum(y):,.2f}")

fig1.update_layout(
    title=dict(text="날짜 컬럼이 있는 테이블 – 연도별 용량 추이", x=0, xanchor='left'),
    xaxis_title="연도",
    yaxis_title="용량 (MB)",
    template="plotly_white",
    xaxis=dict(categoryorder="array", categoryarray=full_order, tickangle=0)
)
display(HTML(pio.to_html(fig1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 5. 날짜 컬럼이 있는 테이블 – 총합 용량 요약표 (MB)
if table_names_1:
    fig_table1 = go.Figure(data=[go.Table(
        header=dict(values=table_names_1, fill_color="lightblue", align="center"),
        cells=dict(values=[[v] for v in totals_1], align="center")
    )])
    fig_table1.update_layout(
        title=dict(text="날짜 컬럼이 있는 테이블 – 총합 용량 (MB)", x=0, xanchor='left'),
        width=len(table_names_1) * 200
    )
    display(HTML(pio.to_html(fig_table1, full_html=False, include_plotlyjs=False)))

    # 5a. 날짜 컬럼이 있는 테이블 – 용량 순 (MB)
    pairs1 = sorted(zip(table_names_1, totals_1),
                    key=lambda x: float(x[1].replace(",", "")),
                    reverse=True)
    names_rank1, vals_rank1 = zip(*pairs1)
    fig_rank1 = go.Figure(data=[go.Table(
        header=dict(values=list(names_rank1), fill_color="lavender", align="center"),
        cells=dict(values=[[v] for v in vals_rank1], align="center")
    )])
    fig_rank1.update_layout(
        title=dict(text="날짜 컬럼이 있는 테이블 – 용량 순 (MB)", x=0, xanchor='left'),
        width=len(names_rank1) * 200
    )
    display(HTML(pio.to_html(fig_rank1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 6. 날짜 컬럼이 있는 테이블 – 월 전체 데이터 표 (MB)
if table_names_1:
    months = sorted({m for content in tables_with_date.values() for m in content.get("month", {})})
    header_month1 = ["월"] + table_names_1
    cells_month1 = [months] + [
        [tables_with_date[tbl].get("month", {}).get(m, "0.000") for m in months]
        for tbl in table_names_1
    ]
    fig_month1 = go.Figure(data=[go.Table(
        header=dict(values=header_month1, align="center"),
        cells=dict(values=cells_month1, align="center")
    )])
    fig_month1.update_layout(
        title=dict(text="날짜 컬럼이 있는 테이블 – 월 전체 데이터 (MB)", x=0, xanchor='left'),
        width=len(header_month1) * 200
    )
    display(HTML(pio.to_html(fig_month1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 7. 날짜 컬럼이 있는 테이블 – 연도 전체 데이터 표 (MB)
if table_names_1:
    years_all = sorted({y for content in tables_with_date.values() for y in content.get("year", {})})
    header_year1 = ["연도"] + table_names_1
    cells_year1 = [years_all] + [
        [tables_with_date[tbl].get("year", {}).get(y, "0.000") for y in years_all]
        for tbl in table_names_1
    ]
    fig_year1 = go.Figure(data=[go.Table(
        header=dict(values=header_year1, align="center"),
        cells=dict(values=cells_year1, align="center")
    )])
    fig_year1.update_layout(
        title=dict(text="날짜 컬럼이 있는 테이블 – 연도 전체 데이터 (MB)", x=0, xanchor='left'),
        width=len(header_year1) * 200
    )
    display(HTML(pio.to_html(fig_year1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 9. 날짜 컬럼이 없는 테이블 – 총합 용량 요약표 (MB)
table_names_2, totals_2 = [], []
for tbl, content in tables_without_date.items():
    year_data = content.get("year", {})
    if not year_data:
        continue
    years2 = sorted([k for k in year_data.keys() if k != "기타"])
    if "기타" in year_data:
        years2.append("기타")
    y2 = [float(year_data[k].replace(",", "")) for k in years2]

    table_names_2.append(tbl)
    totals_2.append(f"{sum(y2):,.2f}")

if table_names_2:
    fig_table2 = go.Figure(data=[go.Table(
        columnwidth=[300, 200],
        header=dict(
            values=["테이블", "총합 용량 (MB)"],
            align="center",
            fill_color="lightgray",
            line_color="black",
            height=40
        ),
        cells=dict(
            values=[table_names_2, totals_2],
            align="center",
            line_color="black",
            height=40
        )
    )])
    fig_table2.update_layout(
        title=dict(text="날짜 컬럼이 없는 테이블 – 총합 용량 (MB)", x=0, xanchor='left'),
        width=600,
        height=600
    )
    display(HTML(pio.to_html(fig_table2, full_html=False, include_plotlyjs=False)))

    # 9a. 날짜 컬럼이 없는 테이블 – 용량 순 (MB)
    pairs2 = sorted(zip(table_names_2, totals_2),
                    key=lambda x: float(x[1].replace(",", "")),
                    reverse=True)
    names_rank2, vals_rank2 = zip(*pairs2)
    fig_rank2 = go.Figure(data=[go.Table(
        columnwidth=[300, 200],
        header=dict(
            values=["테이블", "용량 순 (MB)"],
            align="center",
            fill_color="lavender",
            line_color="black",
            height=40
        ),
        cells=dict(
            values=[list(names_rank2), list(vals_rank2)],
            align="center",
            line_color="black",
            height=40
        )
    )])
    fig_rank2.update_layout(
        title=dict(text="날짜 컬럼이 없는 테이블 – 용량 순 (MB)", x=0, xanchor='left'),
        width=600,
        height=600
    )
    display(HTML(pio.to_html(fig_rank2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 10. 날짜 컬럼이 없는 테이블 – 월 전체 데이터 표 (MB)
if table_names_2:
    months2 = sorted({m for content in tables_without_date.values() for m in content.get("month", {})})
    header_month2 = ["월"] + table_names_2
    cells_month2 = [months2] + [
        [tables_without_date[tbl].get("month", {}).get(m, "0.000") for m in months2]
        for tbl in table_names_2
    ]
    fig_month2 = go.Figure(data=[go.Table(
        header=dict(values=header_month2, align="center"),
        cells=dict(values=cells_month2, align="center")
    )])
    fig_month2.update_layout(
        title=dict(text="날짜 컬럼이 없는 테이블 – 월 전체 데이터 (MB)", x=0, xanchor='left'),
        width=len(header_month2) * 200
    )
    display(HTML(pio.to_html(fig_month2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 11. 날짜 컬럼이 없는 테이블 – 연도 전체 데이터 표 (MB)
if table_names_2:
    years2_all = sorted({y for content in tables_without_date.values() for y in content.get("year", {})})
    header_year2 = ["연도"] + table_names_2
    cells_year2 = [years2_all] + [
        [tables_without_date[tbl].get("year", {}).get(y, "0.000") for y in years2_all]
        for tbl in table_names_2
    ]
    fig_year2 = go.Figure(data=[go.Table(
        header=dict(values=header_year2, align="center"),
        cells=dict(values=cells_year2, align="center")
    )])
    fig_year2.update_layout(
        title=dict(text="날짜 컬럼이 없는 테이블 – 연도 전체 데이터 (MB)", x=0, xanchor='left'),
        width=len(header_year2) * 200
    )
    display(HTML(pio.to_html(fig_year2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 12. 데이터가 없는 테이블 목록 – 5개 그룹으로 나눠서 일렬 배치
empty_tables = data.get("데이터가 없는 테이블 목록", [])
if empty_tables:
    group_size = (len(empty_tables) + 4) // 5
    groups = [empty_tables[i*group_size:(i+1)*group_size] for i in range(5)]
    cells_no_datas = [" ".join(group) for group in groups]
    fig_empty = go.Figure(data=[go.Table(
        columnwidth=[250] * len(cells_no_datas),
        header=dict(
            values=cells_no_datas,
            align="center",
            fill_color="lightgray",
            line_color="black",
            height=60
        )
    )])
    fig_empty.update_layout(
        title=dict(text="데이터가 없는 테이블 목록", x=0, xanchor='left'),
        width=len(cells_no_datas) * 250
    )
    display(HTML(pio.to_html(fig_empty, full_html=False, include_plotlyjs=False)))
else:
    print("\n=== 데이터가 없는 테이블 목록 ===\n없음")


## 추출되지 않는 테이블 명

In [ ]:
import json

# JSON 파일 경로
file_path = '*.json'

# JSON 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 키 목록 정의
part_keys = ["날짜 컬럼이 있는 테이블 명 목록", "날짜 컬럼이 없는 테이블 명 목록", "데이터가 없는 테이블 목록"]
total_key = "총 테이블 명 목록"

# 각 키의 개수 출력
print(f"\n테이블 목록 수")
for key in part_keys + [total_key]:
    count = len(data.get(key, []))
    print(f"- {key}: {count}개")

# 부분 키들의 값 합치기 (중복 제거)
combined_set = set()
for key in part_keys:
    combined_set.update(data.get(key, []))

# 전체 목록 집합
total_set = set(data.get(total_key, []))

# 전체에 없는 값 (불일치 항목)
missing_values = combined_set - total_set

# 결과 출력
print(f"\n'총 테이블 명 목록'에 없는 값: {len(missing_values)}개")
print(missing_values)


## JSON 파일 -> 엑셀

In [ ]:
import json
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
from IPython.display import display, HTML

pio.renderers.default = "notebook_connected"  # 또는 "iframe_connected"

# 1. JSON 불러오기
with open("*.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. 전체 요약 출력
summary_keys = [
    "총 테이블 수",
    "날짜 컬럼이 있는 테이블 수",
    "날짜 컬럼이 없는 테이블 수",
    "데이터가 없는 테이블 수",
    "건너뛴 테이블 수",
    "평균 row length가 0인 테이블 수"
]

print("전체 요약")
for key in summary_keys:
    print(f"{key}: {data.get(key, 0)}")

# ----------------------------------------------------------------
# 3. 날짜 컬럼이 있는 테이블: 라인차트 + 표
tables_with_date = data.get("날짜 컬럼이 있는 테이블", {})
fig1 = go.Figure()
table_names_1 = []
totals_1 = []

for table_name, content in tables_with_date.items():
    year_data = content.get("year", {})
    years = list(year_data.keys())
    values = [float(v.replace(",", "")) for v in year_data.values()]
    
    fig1.add_trace(go.Scatter(x=years, y=values, mode="lines+markers", name=table_name))
    
    table_names_1.append(table_name)
    totals_1.append(f"{sum(values):,.2f}")

fig1.update_layout(
    title="날짜 컬럼이 있는 테이블 - 연도별 추이",
    xaxis_title="연도",
    yaxis_title="용량(MB)",
    template="plotly_white"
)
display(HTML(pio.to_html(fig1, full_html=False, include_plotlyjs='cdn')))

# 표 (가로 출력)
if table_names_1:
    fig_table1 = go.Figure(data=[go.Table(
        header=dict(values=table_names_1, fill_color="lightblue", align="center"),
        cells=dict(values=[[v] for v in totals_1], align="center")
    )])
    fig_table1.update_layout(title="날짜 컬럼이 있는 테이블 - 요약표")
    display(HTML(pio.to_html(fig_table1, full_html=False, include_plotlyjs=False)))

# ----------------------------------------------------------------
# 4. 날짜 컬럼이 없는 테이블: 라인차트 + 표
tables_without_date = data.get("날짜 컬럼이 없는 테이블", {})
fig2 = go.Figure()
table_names_2 = []
totals_2 = []

for table_name, content in tables_without_date.items():
    year_data = content.get("year", {})
    years = list(year_data.keys())
    values = [float(v.replace(",", "")) for v in year_data.values()]
    
    fig2.add_trace(go.Scatter(x=years, y=values, mode="lines+markers", name=table_name))
    
    table_names_2.append(table_name)
    totals_2.append(f"{sum(values):,.2f}")

fig2.update_layout(
    title="날짜 컬럼이 없는 테이블 - 연도별 추이",
    xaxis_title="연도",
    yaxis_title="용량(MB)",
    template="plotly_white"
)
display(HTML(pio.to_html(fig2, full_html=False, include_plotlyjs=False)))

# 표 (가로 출력)
if table_names_2:
    fig_table2 = go.Figure(data=[go.Table(
        header=dict(values=table_names_2, fill_color="lightgray", align="center"),
        cells=dict(values=[[v] for v in totals_2], align="center")
    )])
    fig_table2.update_layout(title="날짜 컬럼이 없는 테이블 - 요약표")
    display(HTML(pio.to_html(fig_table2, full_html=False, include_plotlyjs=False)))

# ----------------------------------------------------------------
# 5. 데이터가 없는 테이블: 표 + print
empty_tables = data.get("데이터가 없는 테이블 목록", [])

if empty_tables:
    fig_table3 = go.Figure(data=[go.Table(
        header=dict(values=empty_tables, fill_color="salmon", align="center"),
        cells=dict(values=[[""] for _ in empty_tables], align="center")
    )])
    fig_table3.update_layout(title="데이터가 없는 테이블 목록")
    display(HTML(pio.to_html(fig_table3, full_html=False, include_plotlyjs=False)))

else:
    print("데이터가 없는 테이블이 없습니다.")

## 특정 키 값을 엑셀로 저장

In [ ]:
import json
import pandas as pd

# 1. JSON 읽기
with open('*.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# 2. ',' 기준으로 분리
raw_list = json_data['총 테이블 명 목록']
rows = []

for line in raw_list:
    parts = [part.strip() for part in line.split(',')]
    for idx, name in enumerate(parts, start=1):
        rows.append({'순번': idx, '테이블명': name})

# 3. 엑셀로 저장
df = pd.DataFrame(rows)
df.to_excel("output.xlsx", index=False)

# 결과값 출력
df
